In [1]:
!pip install --no-build-isolation --index-url https://download.pytorch.org/whl/cu121 \
    torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0

!pip install \
    pyannote.audio==3.3.1 \
    openai-whisper==20231117 \
    transformers==4.34.0 \
    huggingface-hub==0.16.4 \
    tokenizers==0.14.1 \
    ffmpeg-python==0.2.0 \
    librosa==0.10.1 \
    pandas==2.2.2 \
    nltk==3.8.1 \
    seaborn==0.12.2 \
    matplotlib==3.7.1

!pip install speechbrain==0.5.15 \
    resemblyzer==0.1.4 \
    webrtcvad==2.0.10 \
    scikit-learn==1.2.2 \
    soundfile==0.13.1

# spaCy and model (NER fallback, no deps upgrades!)
!pip install -q spacy==3.6.1 --no-deps
!pip install -q https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl --no-deps


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 2.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 85.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 15.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 33.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

In [2]:
import os
import subprocess
import math
import json
import re
import ffmpeg
import librosa
import soundfile as sf
import whisper
import numpy as np
from resemblyzer import VoiceEncoder
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


# **Step-by-Step Workflow**

## **Step 1: Data Ingestion**
#### Loads all video files from a directory, extracts audio using FFmpeg, and saves each audio file as mono 16kHz WAV format for consistent downstream processing.

## **Step 2: Transcription**
#### Uses OpenAI Whisper (offline, local model) to transcribe each audio file into text, returning segments with start and end timestamps.

## **Step 3: Speaker Diarization**
#### Uses Resemblyzer to extract speaker embeddings from each segment and applies Agglomerative Clustering to assign speaker labels, distinguishing between different voices in the audio.

## **Step 4: Time Bucketing**
#### Divides each transcribed segment into fixed 5-second time buckets using the start timestamp, enabling temporal aggregation and analysis.

## **Step 5: Sentiment Analysis**
#### Applies NLTK's VADER sentiment analyzer to classify each text segment as "positive", "negative", or "neutral".

## **Step 6: Named Entity Recognition (NER)**
#### Applies a custom regex-based Named Entity Recognition (NER) method to extract emails, URLs, dates, money, phone numbers, names, organizations, and locations — without using any external ML model.

## **Step 7: Export to CSV**
#### Aggregates all enriched segment information (timestamps, text, speakers, sentiment, entities, word count, and time buckets) into separate CSV files — one per audio — for downstream analysis or visualization.


# **Step 1: Data Ingestion (Video to Audio Conversion)** 
#### * Load video files from the input directory.
#### * Extract audio from each video using ffmpeg.
#### * Convert audio to mono and 16kHz .wav format.
#### * Store the resulting audio files in an output folder (e.g., ./output_audio).

In [3]:
import os
import subprocess

def extract_audio(video_path, output_audio_path):
    """
    Extracts audio from a single video and saves it as mono 16kHz WAV.

    Parameters:
        video_path (str): Path to the input video file.
        output_audio_path (str): Path where the output audio will be saved.

    Returns:
        str: Path to the saved audio file.
    """
    cmd = [
        "ffmpeg", "-i", video_path, "-q:a", "0", "-map", "a",
        "-ac", "1", "-ar", "16000",  # Mono & 16kHz
        output_audio_path, "-y"
    ]
    subprocess.run(cmd, capture_output=True, text=True)
    return output_audio_path

def process_all_videos(input_video_dir):
    """
    Processes all video files in the input directory,
    extracts audio, and saves them to ./output_audio/.

    Parameters:
        input_video_dir (str): Directory containing video files.
    """
    output_audio_dir = "./output_audio"
    os.makedirs(output_audio_dir, exist_ok=True)

    video_extensions = (".mp4", ".mov", ".avi", ".mkv")

    for filename in os.listdir(input_video_dir):
        if filename.lower().endswith(video_extensions):
            video_path = os.path.join(input_video_dir, filename)
            base_name = os.path.splitext(filename)[0]
            output_audio_path = os.path.join(output_audio_dir, f"{base_name}.wav")

            extract_audio(video_path, output_audio_path)
            print(f"✅ Processed: {filename} → {output_audio_path}")

# 🟡 Call this function with your actual input folder containing videos
input_video_directory = "/kaggle/input/videos"
process_all_videos(input_video_directory)  # Change path as needed


✅ Processed: video_convo.mp4 → ./output_audio/video_convo.wav


# **Step 2: Transcription (Using Whisper)**
#### * Load a pre-trained Whisper model (base, small, etc.).
#### * Transcribe each .wav file from the audio output directory.
#### * Store transcription segments with timestamps and text.

In [4]:
def transcribe_all_audio(audio_dir):
    """
    Transcribes all .wav files in a directory using Whisper.

    Parameters:
        audio_dir (str): Path to the directory containing .wav files.

    Returns:
        dict: Mapping from audio filename to list of transcription segments.
    """
    model = whisper.load_model("base")  # You can try 'small' or 'medium' if GPU is available
    transcripts = {}

    for file in os.listdir(audio_dir):
        if file.endswith(".wav"):
            audio_path = os.path.join(audio_dir, file)
            print(f"🔍 Transcribing: {file} ...")
            result = model.transcribe(audio_path)
            transcripts[file] = result["segments"]  # Each segment: dict with 'start', 'end', 'text'
            print(f"✅ Transcribed: {file} → {len(result['segments'])} segments")

    return transcripts

# 🟢 Run this on your extracted audio directory
audio_dir = "./output_audio"
all_transcriptions = transcribe_all_audio(audio_dir)


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 133MiB/s]


🔍 Transcribing: video_convo.wav ...
✅ Transcribed: video_convo.wav → 13 segments


In [5]:
first_audio_file = list(all_transcriptions.keys())[0]  # Get the first file name
first_five_segments = all_transcriptions[first_audio_file][:5]  # First 5 segments

for i, segment in enumerate(first_five_segments, 1):
    print(f"Segment {i}:")
    print(f"Start: {segment['start']}s")
    print(f"End: {segment['end']}s")
    print(f"Text: {segment['text']}")
    print("-" * 40)

Segment 1:
Start: 0.0s
End: 5.28s
Text:  It's a beautiful house. Do you both live here? Yes. Can I get a short house tour? Yeah, mind your shirt.
----------------------------------------
Segment 2:
Start: 7.12s
End: 12.32s
Text:  Welcome to our home. Sat Tichit means true consciousness. This is the house which we will
----------------------------------------
Segment 3:
Start: 12.32s
End: 18.0s
Text:  keep in mind that it enhances our consciousness. These are the waste bottles used to build this mud wall,
----------------------------------------
Segment 4:
Start: 18.0s
End: 21.92s
Text:  the colourful reflection on the backside. Oh wow, you have a hand pump at the end. Yeah.
----------------------------------------
Segment 5:
Start: 23.04s
End: 27.36s
Text:  We use the rain water to write this. Let's go.
----------------------------------------


# **Step 3: Speaker Diarization (Using Resemblyzer + Clustering)**  
#### * Load each audio file and extract its waveform at 16kHz mono.  
#### * For every Whisper transcription segment, compute a speaker embedding using Resemblyzer’s pretrained VoiceEncoder.  
#### * Group similar embeddings using Agglomerative Clustering to identify distinct speakers (e.g., spk_0, spk_1, spk_2).  
#### * Assign a speaker label to each transcription segment based on which cluster its embedding belongs to.  
#### * Optionally merge consecutive segments spoken by the same speaker to form cleaner dialogue turns.


In [6]:
# Diarization via embeddings + clustering (resemblyzer)

audio_dir = "./output_audio"   # same as before
encoder = VoiceEncoder()       # loads pre-trained encoder (small & fast)

def load_wave_16k_mono(path):
    # returns numpy float32 waveform sampled at 16k mono (range -1..1)
    wav, sr = librosa.load(path, sr=16000, mono=True)
    return wav.astype(np.float32), 16000

def embed_segment(waveform, sr, start_s, end_s):
    # waveform is 1D numpy at sr (assumed 16k)
    s = int(start_s * sr)
    e = int(end_s * sr)
    chunk = waveform[s:e]
    if len(chunk) < 160:  # tiny guard
        # pad tiny segments
        chunk = np.pad(chunk, (0, max(0, 160-len(chunk))), mode='constant')
    # resemblyzer expects float32 in range [-1,1]
    try:
        emb = encoder.embed_utterance(chunk)
    except Exception:
        # fallback: if chunk too small or encoder throws, use zero vector
        emb = np.zeros((encoder.embedding_size,))
    return emb

def diarize_file_by_whisper(wav_path, whisper_segments, estimate_speakers=None):
    """
    whisper_segments: list of dicts {'start':float,'end':float,'text':str}
    returns: list of dicts {'start':..,'end':..,'text':..,'speaker':..}
    """
    waveform, sr = load_wave_16k_mono(wav_path)

    # compute embeddings for each whisper segment
    embeddings = []
    valid_segments = []
    for seg in whisper_segments:
        # Skip empty text segments if you prefer:
        # if not seg.get("text","").strip(): continue
        emb = embed_segment(waveform, sr, seg['start'], seg['end'])
        embeddings.append(emb)
        valid_segments.append(seg)

    if len(embeddings) == 0:
        return []

    X = np.vstack(embeddings)

    # Heuristic for number of clusters (adjust if you know speaker count)
    if estimate_speakers is None:
        # choose between 1 and 6 speakers based on segment count
        n_clusters = min(6, max(1, int(max(1, round(len(valid_segments) ** 0.5)))))
        # ensure at least 1
        n_clusters = max(1, n_clusters)
    else:
        n_clusters = int(estimate_speakers)

    # If only 1 segment, label it speaker 0
    if X.shape[0] == 1:
        labels = np.array([0])
    else:
        # Agglomerative clustering; you can tune linkage / affinity if needed
        clustering = AgglomerativeClustering(n_clusters=n_clusters)
        labels = clustering.fit_predict(X)

    # Create enriched segments with speaker labels
    enriched = []
    for seg, lab in zip(valid_segments, labels):
        enriched.append({
            "start_time": round(seg['start'], 2),
            "end_time": round(seg['end'], 2),
            "text": seg['text'],
            "speaker": f"spk_{int(lab)}"
        })

    # Optional: merge adjacent segments with same speaker into longer turns
    merged = []
    for seg in enriched:
        if not merged:
            merged.append(seg.copy())
            continue
        prev = merged[-1]
        if prev['speaker'] == seg['speaker'] and abs(seg['start_time'] - prev['end_time']) <= 0.5:
            # extend previous segment
            prev['end_time'] = seg['end_time']
            prev['text'] = (prev['text'] + " " + seg['text']).strip()
        else:
            merged.append(seg.copy())
    return merged

# Run diarization for all wav files in audio_dir using Whisper segments (all_transcriptions)
all_transcripts_with_speakers = {}
for file in sorted(os.listdir(audio_dir)):
    if not file.endswith(".wav"):
        continue
    wav_path = os.path.join(audio_dir, file)
    base_name = os.path.splitext(file)[0]
    print(f"🔊 Diarizing {file} ...")
    whisper_segs = all_transcriptions.get(file, [])
    if not whisper_segs:
        print("  ⚠️ No whisper transcription found for this audio — skipping.")
        all_transcripts_with_speakers[base_name] = []
        continue

    enriched = diarize_file_by_whisper(wav_path, whisper_segs, estimate_speakers=None)
    all_transcripts_with_speakers[base_name] = enriched

    # print first 5 for quick check
    for i, line in enumerate(enriched[:5], 1):
        print(f"  {i}. [{line['speaker']}] {line['text'][:160]} (Start: {line['start_time']}s, End: {line['end_time']}s)")
    print("-" * 60)


Loaded the voice encoder model on cuda in 0.02 seconds.
🔊 Diarizing video_convo.wav ...
  1. [spk_0]  It's a beautiful house. Do you both live here? Yes. Can I get a short house tour? Yeah, mind your shirt. (Start: 0.0s, End: 5.28s)
  2. [spk_1] Welcome to our home. Sat Tichit means true consciousness. This is the house which we will  keep in mind that it enhances our consciousness. These are the waste  (Start: 7.12s, End: 18.0s)
  3. [spk_0]  the colourful reflection on the backside. Oh wow, you have a hand pump at the end. Yeah. (Start: 18.0s, End: 21.92s)
  4. [spk_0]  We use the rain water to write this. Let's go. (Start: 23.04s, End: 27.36s)
  5. [spk_3]  It's a tiny bridge. (Start: 27.36s, End: 30.08s)
------------------------------------------------------------


# **Step 4: Bucket Transcriptions by Time**
#### * Organize transcribed text into 5-second time buckets.
#### * Each segment is assigned to a bucket_start and bucket_end (e.g., 0–5 sec, 5–10 sec).
#### * Store all segments with their time buckets, speakers, and texts.

In [8]:
import math

def assign_buckets(transcript_segments, bucket_size=5):
    """
    Adds 5-second time bucket information to each transcription segment.

    Parameters:
        transcript_segments (list): List of transcription segments with 'start_time' and 'end_time'.
        bucket_size (int): Size of each time bucket in seconds (default: 5).

    Returns:
        list of dicts: Updated segments with bucket_start and bucket_end.
    """
    for segment in transcript_segments:
        start_bucket = int(math.floor(segment["start_time"] / bucket_size)) * bucket_size
        end_bucket = start_bucket + bucket_size
        segment["bucket_start"] = start_bucket
        segment["bucket_end"] = end_bucket
    return transcript_segments
# Dictionary to store the bucketed transcriptions
bucketed_transcripts = {}
# Assign buckets to all transcription segments
for file_name, segments in all_transcripts_with_speakers.items():
    print(f"Assigning buckets for: {file_name}")
    
    # Assign buckets to the segments
    updated_segments = assign_buckets(segments)
    bucketed_transcripts[file_name] = updated_segments
    
    # Print the first 5 segments 
    print(f"Buckets assigned. First 5 entries:")
    for i, segment in enumerate(updated_segments[:5], start=1):
        print(f"{i}. [Speaker: {segment['speaker']}] {segment['text']} (Start: {segment['start_time']}s, End: {segment['end_time']}s) → Bucket: {segment['bucket_start']}-{segment['bucket_end']}s")

    print("-" * 60)


Assigning buckets for: video_convo
Buckets assigned. First 5 entries:
1. [Speaker: spk_0]  It's a beautiful house. Do you both live here? Yes. Can I get a short house tour? Yeah, mind your shirt. (Start: 0.0s, End: 5.28s) → Bucket: 0-5s
2. [Speaker: spk_1] Welcome to our home. Sat Tichit means true consciousness. This is the house which we will  keep in mind that it enhances our consciousness. These are the waste bottles used to build this mud wall, (Start: 7.12s, End: 18.0s) → Bucket: 5-10s
3. [Speaker: spk_0]  the colourful reflection on the backside. Oh wow, you have a hand pump at the end. Yeah. (Start: 18.0s, End: 21.92s) → Bucket: 15-20s
4. [Speaker: spk_0]  We use the rain water to write this. Let's go. (Start: 23.04s, End: 27.36s) → Bucket: 20-25s
5. [Speaker: spk_3]  It's a tiny bridge. (Start: 27.36s, End: 30.08s) → Bucket: 25-30s
------------------------------------------------------------


# **Step 5: Sentiment Analysis (Using NLTK VADER)**
#### * For each transcribed segment, analyze sentiment using the VADER sentiment analyzer.
#### * Assign a label: positive, negative, or neutral based on compound score.
#### * Add sentiment to the segment metadata.

In [9]:
# Initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    """
    Analyzes sentiment of a given text using VADER SentimentIntensityAnalyzer.

    Parameters:
        text (str): The text to analyze.

    Returns:
        str: Sentiment label: "positive", "negative", or "neutral".
    """
    # Analyze sentiment
    sentiment_score = sia.polarity_scores(text)["compound"]

    # Assign sentiment based on the compound score
    if sentiment_score >= 0.05:
        return "positive"
    elif sentiment_score <= -0.05:
        return "negative"
    else:
        return "neutral"

def apply_sentiment_to_segments(transcript_segments):
    """
    Applies sentiment analysis to each transcription segment.

    Parameters:
        transcript_segments (list): List of transcription segments with 'text' and other info.

    Returns:
        list: Updated transcription segments with sentiment.
    """
    for segment in transcript_segments:
        segment["sentiment"] = analyze_sentiment(segment["text"])
    return transcript_segments

# Dictionary to store sentiment-analyzed transcripts
sentimented_transcripts = {}
for file_name, segments in bucketed_transcripts.items():
    updated = apply_sentiment_to_segments(segments)
    sentimented_transcripts[file_name] = updated

# Apply sentiment analysis for all files
for file_name, segments in bucketed_transcripts.items():
    print(f" Analyzing sentiment for: {file_name}")
    
    # Apply sentiment analysis to segments
    updated_segments_with_sentiment = apply_sentiment_to_segments(segments)
    sentimented_transcripts[file_name] = updated_segments_with_sentiment
    
    # Print the first 5 segments
    print(f"Sentiment analysis complete. First 5 segments:")
    
    for i, entry in enumerate(updated_segments_with_sentiment[:5], start=1):
        print(f"{i}. [Speaker: {entry['speaker']}] {entry['text']} (Sentiment: {entry['sentiment']})")
        print(f"   Start Time: {entry['start_time']}s, End Time: {entry['end_time']}s")
        print(f"   Bucket: {entry['bucket_start']} - {entry['bucket_end']}s")
    print("-" * 60)


 Analyzing sentiment for: video_convo
Sentiment analysis complete. First 5 segments:
1. [Speaker: spk_0]  It's a beautiful house. Do you both live here? Yes. Can I get a short house tour? Yeah, mind your shirt. (Sentiment: positive)
   Start Time: 0.0s, End Time: 5.28s
   Bucket: 0 - 5s
2. [Speaker: spk_1] Welcome to our home. Sat Tichit means true consciousness. This is the house which we will  keep in mind that it enhances our consciousness. These are the waste bottles used to build this mud wall, (Sentiment: positive)
   Start Time: 7.12s, End Time: 18.0s
   Bucket: 5 - 10s
3. [Speaker: spk_0]  the colourful reflection on the backside. Oh wow, you have a hand pump at the end. Yeah. (Sentiment: positive)
   Start Time: 18.0s, End Time: 21.92s
   Bucket: 15 - 20s
4. [Speaker: spk_0]  We use the rain water to write this. Let's go. (Sentiment: neutral)
   Start Time: 23.04s, End Time: 27.36s
   Bucket: 20 - 25s
5. [Speaker: spk_3]  It's a tiny bridge. (Sentiment: neutral)
   Start Time:

# **Step 6: Named Entity Recognition (NER)**
#### * Apply a custom regex-based NER system (no external ML models used).  
#### * Detect structured entities such as:  
   - Emails  
   - URLs  
   - Dates  
   - Money amounts  
   - Phone numbers  
#### * Identify names and organizations using capitalization-based heuristics.  
#### * Extract possible locations using preposition + capitalized word patterns.  
#### * Store all detected entities in a list for each transcription segment.


In [10]:

# === Regex patterns ===
email_re = re.compile(r"[\w\.-]+@[\w\.-]+\.\w{2,}", flags=re.I)
url_re   = re.compile(r"(https?://[^\s,;]+|www\.[^\s,;]+)", flags=re.I)
money_re = re.compile(r"[₹$€]\s?\d+(?:[,\d]*)?(?:\.\d+)?")
date_re  = re.compile(r"\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s+\d{1,2}(?:,\s*\d{4})?)\b", flags=re.I)
phone_re = re.compile(r"\+?\d[\d\-\s()]{6,}\d")

org_endings = {"Ltd","Ltd.","Limited","Inc","Corporation","University","College","Institute","Corp","LLP","PLC","Bank","Foundation","Association"}
loc_preps = {"in","at","from","near","around","to","into","towards","by"}

def find_title_sequences(text, min_len=2, max_len=4):
    tokens = re.findall(r"\b[\w'&\.-]+\b", text)
    spans = []
    n = len(tokens)
    for i in range(n):
        for L in range(min_len, max_len+1):
            if i+L <= n:
                seq = tokens[i:i+L]
                ok = all([t[0].isupper() for t in seq])
                if ok:
                    spans.append(" ".join(seq))
    return list(dict.fromkeys(spans))

def extract_named_entities_no_spacy(text):
    if not text or not isinstance(text, str):
        return []
    ents = []

    # regex entities
    for reg, lab in [(email_re,"EMAIL"),(url_re,"URL"),(money_re,"MONEY"),
                     (date_re,"DATE"),(phone_re,"PHONE")]:
        for m in reg.finditer(text):
            tag = f"{lab}:{m.group(0)}"
            if tag not in ents:
                ents.append(tag)

    # PERSON/ORG heuristics
    title_seqs = find_title_sequences(text)
    for seq in title_seqs:
        if any(tok.rstrip(".,") in org_endings for tok in seq.split()):
            tag = f"ORG:{seq}"
        else:
            tag = f"PERSON:{seq}"
        if tag not in ents:
            ents.append(tag)

    # GPE heuristics
    gpe_matches = re.findall(
        r"\b(" + "|".join(loc_preps) + r")\s+([A-Z][\w\.-]+(?:\s+[A-Z][\w\.-]+)?)",
        text
    )
    for m in gpe_matches:
        city = m[1]
        tag = f"GPE:{city}"
        if tag not in ents:
            ents.append(tag)

    return ents

def apply_ner_to_segments(transcript_segments):
    for segment in transcript_segments:
        segment["named_entities"] = extract_named_entities_no_spacy(segment.get("text",""))
    return transcript_segments


In [12]:
nered_transcripts = {}

for file_name, segments in sentimented_transcripts.items():
    print(f"Applying NER for: {file_name}")

    # Apply NER
    updated_segments = apply_ner_to_segments(segments)
    nered_transcripts[file_name] = updated_segments

    # Print first 5 segments like Step 5
    print("First 5 segments with NER:")
    for idx, seg in enumerate(updated_segments[:5], start=1):
        print(f"{idx}. [Speaker: {seg.get('speaker','')}]")
        print(f"   Text: {seg.get('text','')}")
        print(f"   Named Entities: {seg.get('named_entities', [])}")
        print(f"   Sentiment: {seg.get('sentiment','')}")
        print(f"   Time: {seg.get('start_time')}s → {seg.get('end_time')}s")
        print(f"   Bucket: {seg.get('bucket_start')} - {seg.get('bucket_end')}s")
        print("-" * 60)


Applying NER for: video_convo
First 5 segments with NER:
1. [Speaker: spk_0]
   Text:  It's a beautiful house. Do you both live here? Yes. Can I get a short house tour? Yeah, mind your shirt.
   Named Entities: ['PERSON:Yes Can', 'PERSON:Yes Can I', 'PERSON:Can I']
   Sentiment: positive
   Time: 0.0s → 5.28s
   Bucket: 0 - 5s
------------------------------------------------------------
2. [Speaker: spk_1]
   Text: Welcome to our home. Sat Tichit means true consciousness. This is the house which we will  keep in mind that it enhances our consciousness. These are the waste bottles used to build this mud wall,
   Named Entities: ['PERSON:Sat Tichit']
   Sentiment: positive
   Time: 7.12s → 18.0s
   Bucket: 5 - 10s
------------------------------------------------------------
3. [Speaker: spk_0]
   Text:  the colourful reflection on the backside. Oh wow, you have a hand pump at the end. Yeah.
   Named Entities: []
   Sentiment: positive
   Time: 18.0s → 21.92s
   Bucket: 15 - 20s
---------

# **Step 7: Export to CSV**
#### * Create a dedicated output folder (`./csv_outputs`) to store results for each video.  
#### * Convert each processed segment (containing transcription, speaker, sentiment, entities, and time buckets) into a structured row format.  
#### * Generate a separate CSV file for every input audio/video file, preserving the filename for traceability.  
#### * Each CSV includes:  
   - start_time  
   - end_time  
   - bucket_start / bucket_end  
   - text  
   - sentiment  
   - named_entities  
   - word_count  
   - speaker  
#### * Use safe error-handling to skip broken files and continue processing without interruptions.  
#### * Display a small preview of the first few rows of each generated CSV for verification.  


In [13]:
import os
import pandas as pd
from pathlib import Path
import traceback

def export_per_video_csvs_safe(nered_transcripts, output_dir="./csv_outputs", show_preview=2):
    os.makedirs(output_dir, exist_ok=True)

    if not nered_transcripts:
        print("⚠️ nered_transcripts is empty or falsy. Nothing to export.")
        return

    exported = 0
    for file_name, segments in nered_transcripts.items():
        try:
            if not segments:
                print(f"⚠️ Skipping {file_name}: no segments")
                continue

            rows = []
            for i, segment in enumerate(segments):
                # defensive getters with defaults
                start = segment.get("start_time")
                end   = segment.get("end_time")
                text  = segment.get("text", "")
                sentiment = segment.get("sentiment", "")
                named_entities = segment.get("named_entities", [])
                if not isinstance(named_entities, (list, tuple)):
                    # if it's a string or None, normalize to list
                    if named_entities:
                        named_entities = [str(named_entities)]
                    else:
                        named_entities = []

                row = {
                    "start_time": start,
                    "end_time": end,
                    "bucket_start": int(start // 5) * 5 if isinstance(start, (int,float)) else None,
                    "bucket_end": (int(start // 5) + 1) * 5 if isinstance(start, (int,float)) else None,
                    "text": text,
                    "sentiment": sentiment,
                    "named_entities": ", ".join(named_entities),
                    "word_count": len(str(text).split()),
                    "speaker": segment.get("speaker", "")
                }
                rows.append(row)

            if not rows:
                print(f"⚠️ Skipping {file_name}: rows empty after processing")
                continue

            df = pd.DataFrame(rows)
            output_path = os.path.join(output_dir, f"{file_name}.csv")
            df.to_csv(output_path, index=False)
            exported += 1

            print(f"✅ Exported {file_name} → {output_path} ({len(rows)} rows)")
            if show_preview:
                print(" Preview rows:")
                print(df.head(show_preview).to_dict(orient='records'))
        except Exception as e:
            print(f"❌ Failed exporting {file_name}: {e}")
            traceback.print_exc()

    if exported == 0:
        print("⚠️ No CSVs exported. Check that nered_transcripts contains data in expected format.")
    else:
        print(f"🎉 Exported {exported} CSV file(s) to: {os.path.abspath(output_dir)}")

# example call
export_per_video_csvs_safe(nered_transcripts, output_dir="./csv_outputs", show_preview=2)
# list files created
print("Files in output dir:", list(Path("./csv_outputs").glob("*.csv")))


✅ Exported video_convo → ./csv_outputs/video_convo.csv (10 rows)
 Preview rows:
[{'start_time': 0.0, 'end_time': 5.28, 'bucket_start': 0, 'bucket_end': 5, 'text': " It's a beautiful house. Do you both live here? Yes. Can I get a short house tour? Yeah, mind your shirt.", 'sentiment': 'positive', 'named_entities': 'PERSON:Yes Can, PERSON:Yes Can I, PERSON:Can I', 'word_count': 21, 'speaker': 'spk_0'}, {'start_time': 7.12, 'end_time': 18.0, 'bucket_start': 5, 'bucket_end': 10, 'text': 'Welcome to our home. Sat Tichit means true consciousness. This is the house which we will  keep in mind that it enhances our consciousness. These are the waste bottles used to build this mud wall,', 'sentiment': 'positive', 'named_entities': 'PERSON:Sat Tichit', 'word_count': 35, 'speaker': 'spk_1'}]
🎉 Exported 1 CSV file(s) to: /kaggle/working/csv_outputs
Files in output dir: [PosixPath('csv_outputs/video_convo.csv')]
